# 1 Użyj innego niż powyżej otwartego korpusu danych w j. ang. do wyuczenia algorytmu Word2Vec (CBoW) i zwróć wektor 50 liczb reprezentujących słowo "python". Zapisz model do pliku o nazwie "model-w2v-cbow.bin".

In [1]:
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api
import numpy as np
import spacy 
nlp = spacy.load('en_core_web_sm')
# https://github.com/RaRe-Technologies/gensim-data/blob/master/list.json#L120
data = api.load('fake-news') 

docs=[]
corpus=[]
for index,news in enumerate(data):
  if index==100: #rip computer
    break
  docs.append(nlp(news['text']))
  
for doc in docs: #need to drop stopwords for Word Mover Distance later
  corpus.append([str(token.lemma_).lower() for token in doc if not (token.is_stop or token.is_punct or token.is_space or not token.is_alpha)]) #is_alpha dropped to remove \n\t etc
  

In [2]:

model = Word2Vec(sentences=corpus) #4s

In [3]:
top50=model.wv.most_similar('child',topn=50)
print(model.wv.vocab.keys())

dict_keys(['print', 'pay', 'money', 'interest', 'entire', 'family', 'come', 'need', 'year', 'group', 'steal', 'government', 'million', 'benefit', 'month', 'report', 'numerous', 'case', 'like', 'muslim', 'immigrant', 'commit', 'fraud', 'system', 'way', 'control', 'related', 'attorney', 'general', 'loretta', 'lynch', 'plead', 'fifth', 'administration', 'congressional', 'probe', 'cash', 'payment', 'iran', 'course', 'recall', 'refuse', 'answer', 'question', 'straight', 'corruption', 'talk', 'cover', 'avoid', 'corrupt', 'core', 'investigation', 'lead', 'member', 'congress', 'obama', 'secret', 'effort', 'send', 'billion', 'early', 'accusation', 'accord', 'lawmaker', 'communication', 'obtain', 'washington', 'free', 'present', 'october', 'series', 'approve', 'deliver', 'response', 'assistant', 'peter', 'kadzik', 'bar', 'publicly', 'disclose', 'detail', 'deal', 'american', 'office', 'provide', 'evidence', 'choose', 'inquiry', 'world', 'state', 'sponsor', 'write', 'friday', 'follow', 'letter', '

In [4]:
#zapisz do pliku
model.save('model-w2v-cbow.bin')

# 2 Użyj algorytmu PCA z pakietu scikit-learn celem redukcji z 50 do 2 wymiarów dla każdego słowa ze słownika modelu z zadania 1, a następnie narysuj wykres 2D wizualizujący położenie tych słów w przestrzeni.

In [5]:
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
words = list(model.wv.vocab)
len(words)
pca = PCA(n_components=2, random_state=2137)
seq_arrays = [model.wv.word_vec(word) for word in words]
df = pd.DataFrame(
    seq_arrays
)

print(len(df))
df.head(10)
result = pca.fit_transform(df)
fig, ax = plt.subplots(figsize = (200, 100))
ax.scatter(result[:, 0], result[:, 1])

#Dodawanie adnotacji zajmuje ponad 15 minut na dobrym sprzęcie
for i, txt in enumerate(words):
   x, y = result[i, 0], result[i, 1]
   ax.annotate(txt, [x, y])

1426


# 3 Zapoznaj się z następcą algorytmu Word2Vec czyli algorytmem fastText. Zastosuj go do wybranego przez siebie problemu wektoryzacji tekstu. 

In [6]:

from gensim.models.fasttext import FastText

fast_model=FastText(corpus[1:100])

print(fast_model.wv.most_similar('president'))



[('presidential', 0.9999988079071045), ('presidency', 0.9999967813491821), ('evident', 0.9999962449073792), ('present', 0.9999954700469971), ('incident', 0.9999953508377075), ('statement', 0.9999947547912598), ('content', 0.9999943971633911), ('provider', 0.9999943375587463), ('prevent', 0.9999943375587463), ('state', 0.9999942779541016)]


# 4 Zapoznaj się i wykorzystaj inną metrykę liczenia podobieństwa zdań (np. Word Mover's Distance (WMD)).

In [24]:
#Intentionally using words the model understands. If you start talking about Jalopeno peppers -- while the model knows only news articles -- you'll get funny results.
model.init_sims(replace=True) # normalize word2vec vectors, so they all have equal length
sent_one='Obama is the president'
sent_two='Clinton is the president'
print("Similar: ", )
similar_dist=model.wmdistance(sent_one,sent_two)
print("Similar: ", similar_dist)

dissimilar_one="Was Obama a good president?"
dissimilar_two="Military intervention"
dissimilar_dist=model.wmdistance(dissimilar_one, dissimilar_two)
print("Dissimilar: ", dissimilar_dist)


print("Is distance of similar sentences smaller (closer) than distance of dissimilar sentences? (True=good)", dissimilar_dist>similar_dist)

Similar: 
Similar:  0.013951867644991078
Dissimilar:  0.06815467179139552
Is distance of similar sentences smaller (closer) than distance of dissimilar sentences? (True=good) True
